In [ ]:
cd /home/dsi/mbouchouia/code/git/PRIM/Notebooks/

In [ ]:
import os
os.environ["MKL_NUM_THREADS"] = "40" 
os.environ["NUMEXPR_NUM_THREADS"] = "40" 
os.environ["OMP_NUM_THREADS"] = "40" 
os.nice(10)


In [ ]:
%matplotlib inline

In [ ]:
countDF.mean().mean()

In [ ]:
import sklearn
import itertools
from hyperopt import fmin, tpe, pyll, hp, Trials, STATUS_OK, STATUS_FAIL

from sklearn.metrics import r2_score

from importlib import reload

import numpy as np
import random
import sys
import io
import matplotlib.pyplot as plt
import urllib

import time
import sys
import pymongo
import os
sys.path.append('../scripts/')
sys.path.append('../papers code/DCRNN/')
import Plotting
import folium
import matplotlib
import osmMerger
import CustomUtils
import OsmProcessing
import datetime
import pandas as pd
from tf_cuda_subprocess import *
from   functools import reduce
import seaborn as sns
import models
from mongoConnection import *
sns.set()
results_path = "../images/model results/"
def saveFig():
    if not os.path.isdir(results_path+model_name):
        os.makedirs(results_path+model_name)
    print(results_path+model_name)
    plt.savefig(results_path+model_name+input(),dpi=300,bbox_inches='tight')

import tensorflow as tf

# @RunAsCUDASubprocess(num_gpus=1)
def createSession():
    config = tf.ConfigProto(intra_op_parallelism_threads=32, 
                            inter_op_parallelism_threads=32,
                            allow_soft_placement=True,
                           )

    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    session = tf.Session(config=config,graph=tf.get_default_graph())

    tf.keras.backend.set_session(session)
createSession()
oldModelPlotting=None
from model_comparator import ModelCompare
K = tf.keras.backend
from sklearn import linear_model
import pytz

In [ ]:
speeds = pd.read_pickle("../data/monthsSpeed__0.pckl").join(pd.read_pickle("../data/monthsSpeedMarJul__0.pckl"))

counts = pd.read_pickle('../data/monthsCount__0.pckl').join(pd.read_pickle('../data/monthsCountMarJul__0.pckl'))

speeds.columns = pd.to_datetime(speeds.columns,utc=True).tz_localize("utc").tz_convert(pytz.timezone("Europe/Paris"))
speeds = speeds[speeds.columns[((speeds.columns.time>=datetime.time(15, 0)) & (speeds.columns.time<datetime.time(20, 0) ))]]

counts.columns = pd.to_datetime(counts.columns,utc=True).tz_localize("utc").tz_convert(pytz.timezone("Europe/Paris"))
counts = counts[counts.columns[((counts.columns.time >= datetime.time(15, 0)) & (counts.columns.time<datetime.time(20, 0) ))]]



# speeds = pd.read_pickle("../data/monthsSpeedfullday__0.pckl")

# counts = pd.read_pickle('../data/monthsCountfullday__0.pckl')


mergeResults=pd.read_pickle("../data/mergeResults.pckl")
segmentsMeta=pd.read_pickle("../data/segmentsMeta.pckl")
mergedSegments=pd.read_pickle("../data/mergedSegments.pckl")

data_cleaner = models.DataCleaner(speeds,segmentsMeta,mergeResults,counts,0.8)

In [ ]:
speedsTest = data_cleaner.data[data_cleaner.data.columns[-300:]]
data_cleaner.data=data_cleaner.data[data_cleaner.data.columns[:-300]]
speedDF = data_cleaner.data


countsTest = data_cleaner.counts[data_cleaner.counts.columns[-300:]]
data_cleaner.counts=data_cleaner.counts[data_cleaner.counts.columns[:-300]]

countDF = data_cleaner.counts

In [ ]:
""" we don't want to center using unseen data"""
valid_split = 0.7
true_sequence_length=len(set(pd.to_datetime(speedDF.columns).time))
n_dates=len(set(pd.to_datetime(speedDF.columns).date)) 
train_dates_index = speedDF.columns[:int(valid_split*n_dates)*true_sequence_length]
train_intercept = speedDF[train_dates_index].groupby(pd.to_datetime(train_dates_index).time,axis=1).mean()
intercept_extended_df = pd.concat([train_intercept]*n_dates,axis=1)
intercept_extended_df.columns=speedDF.columns
# centering
speedDF = speedDF-intercept_extended_df

test_intercept = pd.concat([train_intercept]*(len(speedsTest.columns)//true_sequence_length),axis=1)
test_intercept.columns = speedsTest.columns
speedsTest = speedsTest- test_intercept

In [ ]:
(speedDF+intercept_extended_df).mean().plot(use_index=False,figsize=(24,6))

In [ ]:
adjacency_matrix = OsmProcessing.getAdjacencyMatrix(segmentsMeta)
adjacency_matrix = adjacency_matrix.add(adjacency_matrix.T.values,fill_value=0)
adjacency_matrix = OsmProcessing.addLevel(adjacency_matrix,3)
adjacency_matrix = OsmProcessing.mergeAdjacencyMatrix(adjacency_matrix, mergeResults, segmentsMeta)
adjacency_matrix = adjacency_matrix[speedDF.index].loc[speedDF.index]

# Models results

In [ ]:
models_predictions = {
    
}

# Sklearn OLS

In [ ]:
""" warning:  this takes a long time"""


nSegments = len(speedDF)

input_lag, output_lag, sequence_length = 1, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
intercept_data_model = models.DataModel(intercept_extended_df,input_lag,output_lag,sequence_length,valid_split=valid_split)
intercept_data_model.preprocessData()

params        = { "scale_output" : True,"name":"OLS" }

data_model    = models.DataModel( speedDF,    input_lag, output_lag, sequence_length, valid_split = valid_split, **params)

data_model.preprocessData()

x_train_00, y_train_00, x_test_00, y_test_00 = data_model.trainSplit()





x_test=x_test_00.reshape(x_test_00.shape[0:3:2])
y_test=y_test_00

x=x_train_00.reshape(x_train_00.shape[0:3:2])
y=y_train_00


A = [linear_model.LinearRegression() for i in range(nSegments)]


for i in range(nSegments):
    A[i].fit(x, y[:, i])    
    
preds = []

for i in range(nSegments):
    preds.append(A[i].predict(x_test))

preds = np.array(preds)    
    
models_predictions["OLS" ] = data_model.restorePredictionsAsDF(preds.T,'test')


from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
print("MSE:", mean_squared_error(preds.T.flatten(), y_test.flatten()))
print("MAE:", mean_absolute_error(preds.T.flatten(), y_test.flatten()))


In [ ]:
input_lag, output_lag, sequence_length = 1, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
params        = { "scale_output" : True,"name":"OLS" }
test_data_model    = models.DataModel( speedsTest,    input_lag, output_lag, sequence_length, valid_split = 1, **params)
test_data_model.preprocessData()
test_data_x, test_data_y, _, _ = test_data_model.trainSplit()

x_test=test_data_x.reshape(test_data_x.shape[0:3:2])
y_test=test_data_y
preds=[]
for i in range(nSegments):
    preds.append(A[i].predict(x_test))

preds = np.array(preds)    

models_predictions["OLS" ] = test_data_model.restorePredictionsAsDF(preds.T)

print("MSE:", mean_squared_error(preds.T.flatten(), y_test.flatten()))
print("MAE:", mean_absolute_error(preds.T.flatten(), y_test.flatten()))


# Sklearn Lasso

In [ ]:
"""warning:  this takes a long time"""


nSegments = len(speedDF)

input_lag, output_lag, sequence_length = 1, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
intercept_data_model = models.DataModel(intercept_extended_df,input_lag,output_lag,sequence_length,valid_split=valid_split)
intercept_data_model.preprocessData()

params        = { "scale_output" : True,"name":"Lasso-OLS" }

data_model    = models.DataModel( speedDF,    input_lag, output_lag, sequence_length, valid_split = valid_split, **params)

data_model.preprocessData()

x_train_00, y_train_00, x_test_00, y_test_00 = data_model.trainSplit()




x_test=x_test_00.reshape(x_test_00.shape[0:3:2])
y_test=y_test_00

x=x_train_00.reshape(x_train_00.shape[0:3:2])
y=y_train_00


A_lasso = [linear_model.LassoCV(n_jobs=5, cv=5, max_iter=10000) for i in range(nSegments)]


for i in range(nSegments):
    A_lasso[i].fit(x, y[:, i])
    print(i,"n iter:", A_lasso[i].n_iter_)
    
    
preds_lasso = []

for i in range(nSegments):
    preds_lasso.append(A_lasso[i].predict(x_test))

preds_lasso = np.array(preds_lasso)    
    
models_predictions["Lasso-OLS" ] = data_model.restorePredictionsAsDF(preds_lasso.T,'test')


from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print("Avec CV")

print("MSE:", mean_squared_error(preds_lasso.T.flatten(), y_test.flatten()))
print("MAE:", mean_absolute_error(preds_lasso.T.flatten(), y_test.flatten()))


In [ ]:
n_iters = [A_lasso[i].n_iter_ for i in range(556)]

In [ ]:
input_lag, output_lag, sequence_length = 1, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
params        = { "scale_output" : True,"name":"Lasso-OLS" }

test_data_model    = models.DataModel( speedsTest,    input_lag, output_lag, sequence_length, valid_split = 1, **params)
test_data_model.preprocessData()
test_data_x, test_data_y, _, _ = test_data_model.trainSplit()

x_test=test_data_x.reshape(test_data_x.shape[0:3:2])
y_test=test_data_y
preds=[]
for i in range(nSegments):
    preds.append(A_lasso[i].predict(x_test))

preds = np.array(preds)    
models_predictions["Lasso-OLS" ]= test_data_model.restorePredictionsAsDF(preds.T)

print("MSE:", mean_squared_error(preds.T.flatten(), y_test.flatten()))
print("MAE:", mean_absolute_error(preds.T.flatten(), y_test.flatten()))


# TS LASSO


In [ ]:
def split_time(df,t_interval_length = 2,overlapping = True):
    t_index  = np.unique(pd.to_datetime(df.columns).time)
    index_subsets = np.split(t_index,range(t_interval_length,len(t_index),t_interval_length)) if not overlapping else np.array([np.array([t_index[i]  for i in range(j,t_interval_length+j)]) for j in range(len(t_index)-t_interval_length+1)])
    index_subsets = [ pd.Index(pd.to_datetime(df.columns).time).isin(subset) for subset in index_subsets]
    return index_subsets

t_interval_length = 2
overlapping = True
t_index  = np.unique(pd.to_datetime(speedDF.columns).time)

index_subsets =  split_time(speedDF,t_interval_length,overlapping )
all_models=[]
K = tf.keras.backend

A_t = []
losses = []
maes=[]
full_preds,full_y = [],[]
try:
    err_rates
except Exception:
    err_rates =np.zeros(len(index_subsets))

for i,subset in enumerate(index_subsets) : 
    print("subset "+ str(i))
    X = speedDF[speedDF.columns[subset]].copy()
    nSegments = len(X)
    input_lag, output_lag, sequence_length = 1, 1, t_interval_length # speedDF.columns.size
    
    time_intercept = intercept_extended_df[intercept_extended_df.columns[subset]].copy()
    time_intercept_model=models.DataModel(time_intercept, input_lag, output_lag, sequence_length, valid_split = valid_split,scale_output=True)

    params        = { "scale_output" : True}
    
    
    A_t_model    = models.DataModel( X,    input_lag, output_lag, sequence_length, valid_split = valid_split, **params)                                                

    A_t_model.preprocessData()

    A_t_x_train_00, A_t_y_train_00, A_t_x_test_00, A_t_y_test_00 = A_t_model.trainSplit()
 
    x_test=A_t_x_test_00.reshape(A_t_x_test_00.shape[0:3:2])
    y_test=A_t_y_test_00

    x=A_t_x_train_00.reshape(A_t_x_train_00.shape[0:3:2])
    y=A_t_y_train_00


    A_lasso = [linear_model.LassoCV(n_jobs=5, cv=5, max_iter=10000) for i in range(nSegments)]


    for i in range(nSegments):
        A_lasso[i].fit(x, y[:, i])
        print(i,"n iter:", A_lasso[i].n_iter_)

    all_models.append(A_lasso)
    
    

In [ ]:
def split_time(df,t_interval_length = 2,overlapping = True):
    t_index  = np.unique(pd.to_datetime(df.columns).time)
    index_subsets = np.split(t_index,range(t_interval_length,len(t_index),t_interval_length)) if not overlapping else np.array([np.array([t_index[i]  for i in range(j,t_interval_length+j)]) for j in range(len(t_index)-t_interval_length+1)])
    index_subsets = [ pd.Index(pd.to_datetime(df.columns).time).isin(subset) for subset in index_subsets]
    return index_subsets

t_interval_length = 2
overlapping = True
t_index  = np.unique(pd.to_datetime(speedsTest.columns).time)

index_subsets =  split_time(speedsTest,t_interval_length,overlapping )
K = tf.keras.backend

A_t = []
losses = []
maes=[]
full_preds,full_y = [],[]

try:
    err_rates
except Exception:
    err_rates =np.zeros(len(index_subsets))

for i,subset in enumerate(index_subsets) : 
    print("subset "+ str(i))
    X = speedsTest[speedsTest.columns[subset]].copy()
    nSegments = len(X)
    input_lag, output_lag, sequence_length = 1, 1, t_interval_length # speedDF.columns.size
    
    params        = { "scale_output" : True}
    
    
    A_t_model    = models.DataModel( X,    input_lag, output_lag, sequence_length, valid_split = 1, **params)                                                

    A_t_model.preprocessData()

    A_t_x_train_00, A_t_y_train_00, A_t_x_test_00, A_t_y_test_00 = A_t_model.trainSplit()
 
    x_test=A_t_x_train_00.reshape(A_t_x_train_00.shape[0:3:2])
    y_test=A_t_y_train_00

    preds=[]
    A_lasso = all_models[i]
    for j in range(nSegments):
        preds.append(A_lasso[j] .predict(x_test))
    preds = np.array(preds)    
    full_preds.append(A_t_model.restorePredictionsAsDF(preds.T))

    print("MSE:", mean_squared_error(preds.T.flatten(), y_test.flatten()))
    print("MAE:", mean_absolute_error(preds.T.flatten(), y_test.flatten()))


In [ ]:
models_predictions["TS-OLS" ]= pd.concat(full_preds,axis=1).sort_index(axis=1)

# Regime switching lasso

In [ ]:
quarters[13]

In [ ]:
def split_data(speedDF,t=2):
    quarters = np.unique(pd.to_datetime(speedDF.columns).time)
    quarters.sort()
    quarter_split=quarters[t]
    return{ True : speedDF.groupby(pd.to_datetime(speedDF.columns).time<=quarter_split,axis=1).get_group(True),False : speedDF.groupby(pd.to_datetime(speedDF.columns).time>=quarter_split,axis=1).get_group(True)}


split_t=13
left_speed_df,right_speed_df =  split_data(speedDF,13)[True],split_data(speedDF,13)[False]
left_speed_test_df,right_speed_test_df =  split_data(speedsTest,13)[True], split_data(speedsTest,13)[False]

nSegments = len(speedDF)


def lassoModel(df,test_df):
    input_lag, output_lag, sequence_length = 1, 1, len(set(pd.to_datetime(df.columns).time)) # speedDF.columns.size
    data_model    = models.DataModel( df,    input_lag, output_lag, sequence_length, valid_split = valid_split, **params)
    data_model.preprocessData()
    x_train_00, y_train_00, x_test_00, y_test_00 = data_model.trainSplit()
    x_test=x_test_00.reshape(x_test_00.shape[0:3:2])
    y_test=y_test_00

    x=x_train_00.reshape(x_train_00.shape[0:3:2])
    y=y_train_00
    A_lasso = [linear_model.LassoCV(n_jobs=5, cv=5, max_iter=10000, tol=0.001) for i in range(nSegments)]

    for i in range(nSegments):
        A_lasso[i].fit(x, y[:, i])
        print(i,"n iter:", A_lasso[i].n_iter_)
    
    
    
    input_lag, output_lag, sequence_length = 1, 1, len(set(pd.to_datetime(test_df.columns).time)) # speedDF.columns.size

    test_data_model    = models.DataModel( test_df,    input_lag, output_lag, sequence_length, valid_split = 1)
    test_data_model.preprocessData()
    test_data_x, test_data_y, _, _ = test_data_model.trainSplit()

    x_test=test_data_x.reshape(test_data_x.shape[0:3:2])
    y_test=test_data_y
    preds=[]
    for i in range(nSegments):
        preds.append(A_lasso[i].predict(x_test))

    preds = np.array(preds)    
    preds_df= test_data_model.restorePredictionsAsDF(preds.T)

    print("MSE:", mean_squared_error(preds.T.flatten(), y_test.flatten()))
    print("MAE:", mean_absolute_error(preds.T.flatten(), y_test.flatten()))
    
    return A_lasso,preds_df

left_lasso_model,  left_lasso_preds  = lassoModel(left_speed_df,left_speed_test_df)
right_lasso_model, right_lasso_preds = lassoModel(right_speed_df,right_speed_test_df)

In [ ]:
models_predictions["RSM" ]=pd.concat([left_lasso_preds,right_lasso_preds],axis=1).sort_index(axis=1)

In [ ]:
# for model_name, model in models_predictions.items():
#     model.to_pickle("pred results/"+model_name+"preds.pckl")

# Deep MLP (L1, Batch normalization)

In [ ]:
def deepMLPLassoNormalized(data_model,x_train_00,rate):
    
    """
        params        = { "scale_output" : True,"name":"Lasso","scale_max":True }

    """
    
    main_input = tf.keras.layers.Input( x_train_00.shape[1:], name="speed_input")
    
    flatten_input = tf.keras.layers.Lambda(lambda x : x[:,-1,:])(main_input )
#     norm=tf.keras.layers.BatchNormalization()(flatten_input)

    hidden = tf.keras.layers.Dense(nSegments, activation="tanh",kernel_regularizer = tf.keras.regularizers.l1(rate))(flatten_input)
    norm = tf.keras.layers.BatchNormalization()(hidden)

    hidden = tf.keras.layers.Dense(nSegments, activation="tanh",kernel_regularizer = tf.keras.regularizers.l1(rate))(norm)

    output_layer = tf.keras.layers.Dense(nSegments, name="speed_lstm",kernel_regularizer = tf.keras.regularizers.l1(rate))( hidden)
    
    data_model.model = tf.keras.Model(inputs = [main_input], outputs= [output_layer])

    optimizer = tf.keras.optimizers.Adam()
    data_model.model.compile(loss="mse", optimizer=optimizer,metrics=["mse"])


In [ ]:
nSegments = len(speedDF)

input_lag, output_lag, sequence_length = 1, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
intercept_data_model = models.DataModel(intercept_extended_df,input_lag,output_lag,sequence_length,valid_split=valid_split)
intercept_data_model.preprocessData()

params        = { "scale_output" : True,"name":"MLP","scale_max":True }

data_model    = models.DataModel( speedDF,    input_lag, output_lag, sequence_length, valid_split = valid_split, **params)

data_model.preprocessData()

x_train_00, y_train_00, x_test_00, y_test_00 = data_model.trainSplit()

count_data = models.DataModel( countDF, input_lag, output_lag, sequence_length, valid_split = valid_split)

count_train_00, _, count_test_00, _ = count_data.trainSplit()

tf.keras.backend.clear_session()

deepMLPLassoNormalized(data_model, x_train_00, rate = 2e-6)

reduce_lr  = tf.keras.callbacks.ReduceLROnPlateau( monitor = 'val_mean_squared_error', factor = 0.5, verbose=1, patience=5, cooldown=5)
early_stop = tf.keras.callbacks.EarlyStopping( monitor='val_mean_squared_error', patience = 15, restore_best_weights=True)

batch_size = 8

modelHist = data_model.model.fit([x_train_00], [y_train_00], validation_data=([x_test_00],[y_test_00]), batch_size=batch_size,epochs=400, callbacks=[reduce_lr,early_stop], verbose=0)
print(data_model.mse(data_model.predict('train'), y_train_00), data_model.mse(data_model.predict('test'),y_test_00))
print(data_model.mae(data_model.predict('train'), y_train_00), data_model.mae(data_model.predict('test'),y_test_00))


In [ ]:
input_lag, output_lag, sequence_length = 1, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
params        = { "scale_output" : True,"name":"MLP","scale_max":True }

test_data_model    = models.DataModel( speedsTest,    input_lag, output_lag, sequence_length, valid_split = 1, **params)
test_data_model.preprocessData()
test_data_x, test_data_y, _, _ = test_data_model.trainSplit()

print(data_model.mse(data_model.predict('custom',x=test_data_x), test_data_y))
print(data_model.mae(data_model.predict('custom',x=test_data_x), test_data_y))

In [ ]:
# models_predictions[data_model.name] = data_model.restorePredictionsAsDF(data_model.predict('test'),'test')
models_predictions[data_model.name] = test_data_model.restorePredictionsAsDF(data_model.predict('custom',x=test_data_x))

In [ ]:
# alpha_max = 0.1
# eps=1e-5
# n_alphas=50
# alphas =np.logspace(np.log10(alpha_max * eps), np.log10(alpha_max),
#                        num=n_alphas)[::-1]
# mlpResults=[]
# for alpha in alphas :
#     nSegments = len(speedDF)

#     input_lag, output_lag, sequence_length = 1, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
#     intercept_data_model = models.DataModel(intercept_extended_df,input_lag,output_lag,sequence_length,valid_split=valid_split)
#     intercept_data_model.preprocessData()

#     params        = { "scale_output" : True,"name":"MLP","scale_max":True }

#     data_model    = models.DataModel( speedDF,    input_lag, output_lag, sequence_length, valid_split = valid_split, **params)

#     data_model.preprocessData()

#     x_train_00, y_train_00, x_test_00, y_test_00 = data_model.trainSplit()

#     count_data = models.DataModel( countDF, input_lag, output_lag, sequence_length, valid_split = valid_split)

#     count_train_00, _, count_test_00, _ = count_data.trainSplit()

#     tf.keras.backend.clear_session()

#     # deepMLPLassoNormalized(data_model, x_train_00, rate = 0.54e-5)
# #     deepMLPLassoNormalized(data_model, x_train_00, rate = 0.24e-5)
#     deepMLPLassoNormalized(data_model, x_train_00, rate = alpha)

#     reduce_lr  = tf.keras.callbacks.ReduceLROnPlateau( monitor = 'val_mean_squared_error', factor = 0.5, verbose=1, patience=5, cooldown=5)
#     early_stop = tf.keras.callbacks.EarlyStopping( monitor='val_mean_squared_error', patience = 15, restore_best_weights=True)

#     batch_size = 8

#     modelHist = data_model.model.fit([x_train_00], [y_train_00], validation_data=([x_test_00],[y_test_00]), batch_size=batch_size,epochs=400, callbacks=[reduce_lr,early_stop], verbose=0)
#     print(data_model.mse(data_model.predict('train'), y_train_00), data_model.mse(data_model.predict('test'),y_test_00))
#     print(data_model.mae(data_model.predict('train'), y_train_00), data_model.mae(data_model.predict('test'),y_test_00))
#     mlpResults.append((alpha,data_model.mse(data_model.predict('test'),y_test_00),data_model.mae(data_model.predict('test'),y_test_00)))
# data_model.model.summary()

# LSTM

In [ ]:
def lstmLassoNormalized(data_model,x_train_00,rate):
    
    """

    """
    
    main_input = tf.keras.layers.Input( x_train_00.shape[1:], name="speed_input")
    

    hidden = tf.keras.layers.LSTM(200,activation="tanh",kernel_regularizer = tf.keras.regularizers.l1(rate))(main_input)


    output_layer=tf.keras.layers.Dense(nSegments, name="speed_lstm",kernel_regularizer = tf.keras.regularizers.l1(rate))(hidden)
    
    data_model.model = tf.keras.Model(inputs = [main_input], outputs= [output_layer])

    optimizer = tf.keras.optimizers.Adam()
    data_model.model.compile(loss="mse", optimizer=optimizer,metrics=["mse"])


In [ ]:
nSegments = len(speedDF)

input_lag, output_lag, sequence_length = 2, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
intercept_data_model = models.DataModel(intercept_extended_df,input_lag,output_lag,sequence_length,valid_split=valid_split)
intercept_data_model.preprocessData()

params        = { "scale_output" : True,"name":"FC-"+str(input_lag)+"LSTM","scale_max":True }

data_model    = models.DataModel( speedDF,    input_lag, output_lag, sequence_length, valid_split = valid_split, **params)

data_model.preprocessData()

x_train_00, y_train_00, x_test_00, y_test_00 = data_model.trainSplit()

count_data = models.DataModel( countDF, input_lag, output_lag, sequence_length, valid_split = valid_split)

count_train_00, _, count_test_00, _ = count_data.trainSplit()

tf.keras.backend.clear_session()

#     lstmLassoNormalized(data_model,x_train_00,rate=0.08e-5)
lstmLassoNormalized(data_model,x_train_00,rate=8e-7)

reduce_lr  = tf.keras.callbacks.ReduceLROnPlateau( monitor = 'val_mean_squared_error', factor = 0.5, verbose=1, patience=5, cooldown=5)
early_stop = tf.keras.callbacks.EarlyStopping( monitor='val_mean_squared_error', patience = 20, restore_best_weights=True)

batch_size = 8

modelHist = data_model.model.fit([x_train_00], [y_train_00], validation_data=([x_test_00],[y_test_00]), batch_size=batch_size,epochs=400, verbose=1,callbacks=[reduce_lr,early_stop])
print(data_model.mse(data_model.predict('train'), y_train_00), data_model.mse(data_model.predict('test'),y_test_00))
print(data_model.mae(data_model.predict('train'), y_train_00), data_model.mae(data_model.predict('test'),y_test_00))


In [ ]:
input_lag, output_lag, sequence_length = 2, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
params        = { "scale_output" : True,"name":"FC-"+str(input_lag)+"LSTM","scale_max":True }

test_data_model    = models.DataModel( speedsTest,    input_lag, output_lag, sequence_length, valid_split = 1, **params)
test_data_model.preprocessData()
test_data_x, test_data_y, _, _ = test_data_model.trainSplit()

print(data_model.mse(data_model.predict('custom',x=test_data_x), test_data_y))
print(data_model.mae(data_model.predict('custom',x=test_data_x), test_data_y))

In [ ]:
models_predictions[data_model.name] = test_data_model.restorePredictionsAsDF(data_model.predict('custom',x=test_data_x))

In [ ]:
# alpha_max = 5e-7
# eps=2e-1
# n_alphas=50
# alphas =np.logspace(np.log10(alpha_max * eps), np.log10(alpha_max),
#                        num=n_alphas)[::-1]
# lstmResults=[]
# for alpha in alphas :
    
#     nSegments = len(speedDF)

#     input_lag, output_lag, sequence_length = 2, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
#     intercept_data_model = models.DataModel(intercept_extended_df,input_lag,output_lag,sequence_length,valid_split=valid_split)
#     intercept_data_model.preprocessData()

#     params        = { "scale_output" : True,"name":"FC-"+str(input_lag)+"LSTM","scale_max":True }

#     data_model    = models.DataModel( speedDF,    input_lag, output_lag, sequence_length, valid_split = valid_split, **params)

#     data_model.preprocessData()

#     x_train_00, y_train_00, x_test_00, y_test_00 = data_model.trainSplit()

#     count_data = models.DataModel( countDF, input_lag, output_lag, sequence_length, valid_split = valid_split)

#     count_train_00, _, count_test_00, _ = count_data.trainSplit()

#     tf.keras.backend.clear_session()

# #     lstmLassoNormalized(data_model,x_train_00,rate=0.08e-5)
#     lstmLassoNormalized(data_model,x_train_00,rate=alpha)

#     reduce_lr  = tf.keras.callbacks.ReduceLROnPlateau( monitor = 'val_mean_squared_error', factor = 0.5, verbose=1, patience=5, cooldown=5)
#     early_stop = tf.keras.callbacks.EarlyStopping( monitor='val_mean_squared_error', patience = 20, restore_best_weights=True)

#     batch_size = 8

#     modelHist = data_model.model.fit([x_train_00], [y_train_00], validation_data=([x_test_00],[y_test_00]), batch_size=batch_size,epochs=400, verbose=0,callbacks=[reduce_lr,early_stop])
#     print(data_model.mse(data_model.predict('train'), y_train_00), data_model.mse(data_model.predict('test'),y_test_00))
#     print(data_model.mae(data_model.predict('train'), y_train_00), data_model.mae(data_model.predict('test'),y_test_00))
#     lstmResults.append((alpha,data_model.mse(data_model.predict('test'),y_test_00),data_model.mae(data_model.predict('test'),y_test_00)))
# data_model.model.summary()
# pd.DataFrame(lstmResults)

### baseline comparison 

In [ ]:
input_lag = 5

In [ ]:

params        = {"name":"PO"}

lastValue_data_model = models.DataModel( speedDF,    input_lag, output_lag, sequence_length, valid_split = valid_split, **params )
lastValue_data_model.preprocessData()

model = models.BaseModels("lastValue",historic_data = speedDF[speedDF.columns[:int(valid_split*n_dates)*true_sequence_length]])
lastValue_data_model.model=model




lastValue_model_plotting = models.ModelPlots(lastValue_data_model, data_cleaner)


In [ ]:

params        = {"name" : "HA"}

timehistoric_data_model    = models.DataModel( speedDF,    input_lag, output_lag, sequence_length, valid_split = valid_split, **params )
timehistoric_data_model.preprocessData()

model =models.BaseModels("timeHistoric",historic_data=speedDF[speedDF.columns[:int(valid_split*n_dates)*true_sequence_length]])
timehistoric_data_model.model=model


In [ ]:
AR_data_models ={}
for i in range(1,input_lag+1):
    params        = {"name":"AR "+str(i)}
    AR_data_models[i]=models.DataModel( speedDF,    i, output_lag, sequence_length, valid_split = valid_split, **params )
    AR_data_models[i].preprocessData()
    model =models.BaseModels("AR5",historic_data=speedDF[speedDF.columns[:int(valid_split*n_dates)*true_sequence_length]],lag=i)
    AR_data_models[i].model = model

    
    input_lag, output_lag, sequence_length = i, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
    params        = { "scale_output" : True }
    test_data_model    = models.DataModel( speedsTest,    input_lag, output_lag, sequence_length, valid_split = 1, **params)
    test_data_model.preprocessData()
    test_data_x, test_data_y, _, _ = test_data_model.trainSplit()
    
    models_predictions[AR_data_models[i].name]=test_data_model.restorePredictionsAsDF(AR_data_models[i].predict('custom',x=test_data_x))

In [ ]:
all_params=[AR_data_models[5].model.models[section].params for section in np.arange(nSegments)]
all_params=pd.DataFrame(all_params)

In [ ]:
all_params.min()

In [ ]:
all_params.mean()

In [ ]:
section = 497#np.random.randint(0,556)
plt.figure(figsize=(16,4))
plt.subplot(121)
plt.plot(AR_data_models[5].model.models[section].Y[0:10])
plt.plot(AR_data_models[1].model.models[section].predict(start=speedDF.columns[5], end=speedDF.columns[15*1]))
plt.plot(AR_data_models[2].model.models[section].predict(start=speedDF.columns[5], end=speedDF.columns[15*1]))
plt.plot(AR_data_models[3].model.models[section].predict(start=speedDF.columns[5], end=speedDF.columns[15*1]))
plt.plot(AR_data_models[4].model.models[section].predict(start=speedDF.columns[5], end=speedDF.columns[15*1]))
plt.plot(AR_data_models[5].model.models[section].predict(start=speedDF.columns[5], end=speedDF.columns[15*1]))
plt.subplot(122)
plt.plot(AR_data_models[5].model.models[section].params)
plt.ylim(-1,1)

In [ ]:
all_baseline_models = [ timehistoric_data_model, lastValue_data_model]

In [ ]:
# for model in all_baseline_models:
#     print(model.name)
#     models_predictions[model.name]=model.restorePredictionsAsDF(model.predict('test'),'test')

input_lag, output_lag, sequence_length = 5, 1, len(set(pd.to_datetime(speedDF.columns).time)) # speedDF.columns.size
params        = { "scale_output" : True }
test_data_model    = models.DataModel( speedsTest,    input_lag, output_lag, sequence_length, valid_split = 1, **params)
test_data_model.preprocessData()
test_data_x, test_data_y, _, _ = test_data_model.trainSplit()

for model in all_baseline_models:
    print(model.name)
    models_predictions[model.name]=test_data_model.restorePredictionsAsDF(model.predict('custom',x=test_data_x))

In [ ]:
file_names =os.listdir("pred results/")[1:]

In [ ]:
models_predictions = {x[:-10]:pd.read_pickle("pred results/"+x) for x in file_names}

In [ ]:
sub_dict_models_predictions = {k:v for k,v in models_predictions.items() if k in models_predictions.keys()}
# sub_dict_models_predictions = {k:v for k,v in models_predictions.items() if k in ['Lasso-OLS', 'OLS', 'MLP', 'FC-2LSTM', 'TS-OLS', 'HA', 'PO', 'AR 1', 'AR 2']}
sub_dict_models_predictions = {k:v for k,v in models_predictions.items() if k in ['Lasso-OLS', 'mix_preds','RSM','TS-OLS','RSM_12_15']}


In [ ]:
models_predictions.keys()

### results table

In [ ]:
sub_dict_models_predictions = {k:v for k,v in models_predictions.items() if k in models_predictions.keys()}

import model_comparator
model_comparator=reload(model_comparator)
input_lag = 5
output_lag=1
true_model=models.DataModel(speedsTest,input_lag,output_lag,20,valid_split=1)
true_model.preprocessData()
true_values=true_model.restorePredictionsAsDF(true_model.trainSplit()[1],'train')
mc=model_comparator.ModelCompare(sub_dict_models_predictions.copy(),
                                 true_values=true_values)
# mc.time_index=pd.to_datetime(mc.time_index).tz_localize("utc").tz_convert(pytz.timezone("Europe/Paris"))
res_table=mc.comparisonTable().round(2)
res_table=res_table.sort_index()

In [ ]:
print(res_table.T.to_latex())
res_table

In [ ]:
sub_dict_models_predictions = {k:v for k,v in models_predictions.items() if k in ['Lasso','RS-Lasso','TS-Lasso',"MLP","FC-2LSTM",'AR 3','HA']}

import model_comparator
model_comparator=reload(model_comparator)
input_lag = 5
output_lag=1
true_model=models.DataModel(speedsTest,input_lag,output_lag,20,valid_split=1)
true_model.preprocessData()
true_values=true_model.restorePredictionsAsDF(true_model.trainSplit()[1],'train')
mc=model_comparator.ModelCompare(sub_dict_models_predictions.copy(),
                                 true_values=true_values)

In [ ]:
sub_dict_models_predictions.keys()

In [ ]:

# plt.figure(figsize=(18,6))
# mc.sample_predictions(seg_ids=[speedsTest.index[6]],day_ids=pd.Series(speedsTest.columns).dt.date.unique()[-1:],intercept=test_intercept)

In [ ]:
# plt.figure(figsize=(18,18))
# mc.plotAverageError(smooth_sigma=1,subplot=True)

In [ ]:
plt.figure(figsize=(8,4))
mc.fontsize=20
mc.futurError()
plt.legend(ncol=4,loc=(0.0,1.01),fontsize=13)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig("average_daily_error.png",dpi=900,pad_inches=0,bbox_inches="tight")


In [ ]:
plt.figure(figsize=(8,4))
mc.plotTimeError()
plt.legend(ncol=1,fontsize=12)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig("average_time_error.png",dpi=900,pad_inches=0,bbox_inches="tight")

In [ ]:
# plt.figure(figsize=(18,18))
# mc.plotAveragePrediction(subplot=True,intercept=test_intercept)

In [ ]:
# plt.figure(figsize=(18,18))

# mc.plotDiscreteSpeedError(test_intercept)

In [ ]:
# days_var = speedDF.abs().mean()
# days_var=speedDF.abs().groupby(pd.to_datetime(speedDF.columns).date,axis=1).mean().mean()

In [ ]:
# plt.figure(figsize=(22,10))
# plt.plot(days_var.values)
# plt.xticks(np.r_[:len(days_var.index)-1:66j].astype(int),days_var.index[np.r_[:len(days_var.index)-1:66j].astype(int)],rotation=90);

In [ ]:
# plt.figure(figsize=(18,18))
# mc.errorSegmentStdOrder(2)

In [ ]:
import sklearn.cluster

"""
Kmeans clustering of segments
"""
grouped_data=(speedDF+intercept_extended_df).T.groupby(pd.to_datetime(speedDF.columns).time,axis=0).describe()

raw_segs_desc=grouped_data.stack().T
# raw_segs_desc=raw_segs_desc[raw_segs_desc.columns[raw_segs_desc.columns.get_level_values(1)=='std']]
segs_desc=raw_segs_desc.apply(sklearn.preprocessing.minmax_scale,feature_range=(-1,1))

lbls=sklearn.cluster.k_means(segs_desc,3,n_init=100)[1]

mean_ordered_groups_index=(speedDF + intercept_extended_df).groupby(lbls).mean().mean(axis=1).sort_values().index

group_labels=["low","medium_low","medium_high","high"]

group_dict = dict(zip(mean_ordered_groups_index,group_labels))
lbls=[group_dict[x] for x in lbls]

plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
pd.Series(dict(zip(*np.unique(lbls,return_counts = True))),name="segment per group").plot.bar(ax=plt.gca())
plt.ylabel("#Roads")

"""
average day timeseries
"""
plt.subplot(1,2,2)
(speedDF+intercept_extended_df).groupby(pd.to_datetime(speedDF.columns).time,axis=1).mean().groupby(lbls).mean().T.plot(subplots=False,use_index=False,ax=plt.gca())
xlbls=sorted(np.unique(pd.to_datetime(speedDF.columns).time))
plt.xticks(np.arange(len(xlbls)),xlbls,rotation=90);
plt.ylabel("Speed (km/h)")
plt.savefig("roads_clusters.png", dpi=900, bbox_inches="tight")
"""
full timeseries
"""
# plt.subplot(1,3,3)
# (speedDF+intercept_extended_df).groupby(lbls).mean().T.plot(subplots=False,use_index=False,ax=plt.gca())
# plt.title("full timeseries")
groups_seg_index = speedDF.groupby(lbls).groups

In [ ]:
sub_dict_models_predictions = {k:v for k,v in models_predictions.items() if k in ['Lasso','RS-Lasso','TS-Lasso',"MLP","FC-2LSTM",'AR 3','HA']}

input_lag = 5
true_model=models.DataModel(speedsTest,input_lag,output_lag,20,valid_split=1)
true_model.preprocessData()
true_values=true_model.restorePredictionsAsDF(true_model.trainSplit()[1],'train')

mc=model_comparator.ModelCompare(sub_dict_models_predictions.copy(),
                                 true_values=true_values,
                                 segments_index=groups_seg_index['medium_low'])
print(mc.comparisonTable().round(2).T.to_latex())
mc.comparisonTable().round(2).T
res_table=mc.comparisonTable().round(2)
res_table=res_table.sort_index()

In [ ]:
print(res_table.T.to_latex())
res_table

In [ ]:

plt.figure(figsize=(8,4))
mc.fontsize=20

mc.futurError()
plt.legend(ncol=4,loc=(0.0,1.01),fontsize=13)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig("average_daily_error_cluster.png",dpi=900,pad_inches=0,bbox_inches="tight")

In [ ]:
plt.figure(figsize=(8,4))
mc.plotTimeError()
plt.legend(ncol=1,fontsize=12)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.savefig("average_time_error_cluster.png",dpi=900,pad_inches=0,bbox_inches="tight")

# Special events

In [ ]:
input_lag=5
true_model=models.DataModel(speedDF,input_lag,output_lag,20,valid_split=0.7)
true_model.preprocessData()
true_values=true_model.restorePredictionsAsDF(true_model.trainSplit()[3],'test')
mc=model_comparator.ModelCompare(sub_dict_models_predictions.copy(),
                                 true_values=true_values[true_values[true_values.columns[pd.to_datetime(true_values.columns).date==datetime.datetime(2019,6,7).date()]].columns])
mc.comparisonTable()
print(mc.comparisonTable())

In [ ]:
mc.plotTimeError()


In [ ]:
def get_kepler_plotable_data(segmentsMeta,speedDF,At):
    submeta = segmentsMeta.copy()
    submeta = submeta.set_index("segmentID").loc[speedDF.index]
    loc_idx = submeta['loc'].apply(lambda x : x['coordinates'][len(x['coordinates'])//2][0]).values
    df =pd.DataFrame(np.array([loc_idx]*len(speedDF)).flatten(),columns=['long_r'])
    df = df.assign(long_l=pd.Series(loc_idx , index=range(0,nSegments**2,nSegments)))
    df =df.assign(segmentId_r = np.array([speedDF.index.values]*len(speedDF)).flatten())

    loc_idx=submeta['loc'].apply(lambda x : x['coordinates'][len(x['coordinates'])//2][1]).values
    
    df = df.assign(lat_r=np.array([loc_idx]*len(speedDF)).flatten() )
    df = df.assign(lat_l=pd.Series(loc_idx,index=range(0,nSegments**2,nSegments)))
    df = df.assign(segmentId_l=pd.Series(speedDF.index.values,index=range(0,nSegments**2,nSegments)))
    df = df.ffill()

    df=df.assign( rate = At.flatten() )
    return df

In [ ]:
A=np.array([A.coef_ for A in A_lasso])

In [ ]:
df = get_kepler_plotable_data(segmentsMeta,speedDF,abs(A))

# df = df.assign( adjacency = adjacency_matrix.values.flatten() )

# df = df.assign( diffusion = diffusion_matrix.values.flatten() )

df.to_csv("../data/segment_flow.csv")

In [ ]:
plt.figure(figsize=(18,9))
plt.subplot(121)
plt.imshow(A)
plt.subplot(122)
plt.imshow(adjacency_matrix>1)

In [ ]:
import re

In [ ]:
active_sum   = abs(A).sum(axis=0)

In [ ]:
active_count = (abs(A)!=0).sum(axis=0)

In [ ]:
active_mean  = active_sum/((active_count==0) + active_count)

In [ ]:
plt.plot(sorted(active_mean))
plt.figure()
plt.plot(sorted(active_sum))
plt.figure()
plt.plot(sorted(active_count))

In [ ]:
def createMap(A_l,log=False,name="map"):

    if log :
        A_l=sklearn.preprocessing.minmax_scale(np.log1p(A_l))
    else:
        A_l=sklearn.preprocessing.minmax_scale(A_l)

    color_coefs=speedDF.assign(color=A_l)['color']
    color_coefs=color_coefs.apply(lambda x: matplotlib.colors.rgb2hex(plt.cm.hot(x)))
    color_coefs=data_cleaner.mergedIndex.replace(color_coefs).replace("^[^#]+","#000000",regex=True).values

    fmap = Plotting.getFoliumMap()
    layer= Plotting.plotRoads(segmentsMeta,fmap=fmap,colors=color_coefs,inverseIndexes=data_cleaner.mergedIndex,weight=4)
    Plotting.stackHistotyLayers([layer],fmap)
    
    fmap.save(name+".html")

In [ ]:
createMap(active_mean)

In [ ]:
createMap(active_sum, log=True, name="active_sum")
plt.figure(figsize=(8,8))
plt.imshow(np.log1p(active_sum.reshape(1,-1)).T,  cmap=plt.cm.hot)
cbar=plt.colorbar()
cbar.ax.set_yticklabels(np.expm1( cbar.get_ticks()).round(2));
cbar.set_label("sum of active coefs")

In [ ]:
createMap(active_count)

In [ ]:
segs_std=(speedDF[train_dates_index]+intercept_extended_df[train_dates_index]).std(axis=1)
createMap(segs_std,name="segstd")

plt.figure(figsize=(8,8))
plt.imshow(segs_std.values.reshape(1,-1).T,  cmap=plt.cm.hot)
cbar=plt.colorbar()
cbar.set_label("STD")

In [ ]:
segs_std.max()

In [ ]:
sub_dict_models_predictions.keys()

In [ ]:


day_ids = np.unique(pd.to_datetime(mc.true_values.columns).date)[:2]

plt.figure(figsize=(12,12))
for i,seg_i in enumerate(seg_ids):
    
    for j,day_i in enumerate(day_ids):
        plt.subplot(len(seg_ids),len(day_ids),i*len(day_ids)+j+1)
        time_ids  = mc.true_values.columns[pd.Series(mc.true_values.columns).dt.date.isin([day_i]).values]
        true_vals = (mc.true_values+intercept_extended_df).loc[seg_i][time_ids]
        Lasso_vals = (sub_dict_models_predictions['Lasso-OLS']+intercept_extended_df).loc[seg_i][time_ids]
        plt.plot(true_vals.values)
        plt.plot(Lasso_vals.values)
        print(len(true_vals.values),len(Lasso_vals.values))
        myslice=slice(None,None,2)
        plt.xticks(np.arange(len(true_vals))[myslice],pd.Series(true_vals.index).dt.time.astype(str).values[myslice],rotation=90)
plt.tight_layout()

In [ ]:
sequence_length=20
all_results_df=pd.read_pickle('allresults_vf5.pckl')
quarters = np.unique(pd.to_datetime(speedDF.columns).time)
quarters.sort()
A1_results=all_results_df[['loss','t','left']][all_results_df['left']==True]

A1_results=A1_results.assign(weighted_loss_A1=A1_results.apply(lambda x: (x['t'])*x['loss'] ,axis=1))

A2_results=all_results_df[['loss','t','left']][all_results_df['left']==False]

A2_results=A2_results.assign(weighted_loss_A2=A2_results.apply(lambda x: ((sequence_length-1)-x['t'])*x['loss'] ,axis=1))

plt.plot(((A1_results.groupby('t').min()['weighted_loss_A1']+A2_results.groupby('t').min()['weighted_loss_A2'])/(sequence_length-1)))
plt.xticks(range(len(quarters)),quarters,rotation=90)
plt.axhline(115.29880658445208,c='r')

In [ ]:
L_A = pd.read_pickle("left_lasso_model.pckl")
R_A = pd.read_pickle("right_lasso_model.pckl")

In [ ]:
defultdf=pd.DataFrame(index=range(556),columns=range(556))
R_A=R_A.unstack()
defultdf.update(R_A)
R_A =defultdf

In [ ]:
defultdf=pd.DataFrame(index=range(556),columns=range(556))
L_A=L_A.unstack()
defultdf.update(L_A)
L_A =defultdf

In [ ]:
(R_A>0).sum().sum()

In [ ]:
(L_A>0).sum().sum()

In [ ]:
((R_A>0)&(L_A>0)).sum().sum()

In [ ]:
(((R_A>0)|(L_A>0))&(A>0)).sum().sum()

In [ ]:
(A>0).sum().sum()

In [ ]:
(((R_A>0))&(L_A>0)).sum().sum()

In [ ]:
(((R_A>0))&(A>0)).sum().sum()

In [ ]:
(((L_A>0))&(A>0)).sum().sum()

In [ ]:
sklearn.metrics.normalized_mutual_info_score((R_A>0).values.flatten(),(A>0).flatten())

In [ ]:
sklearn.metrics.normalized_mutual_info_score((L_A>0).values.flatten(),(A>0).flatten())

In [ ]:
nmis=[sklearn.metrics.adjusted_mutual_info_score((L_A>tresh).values.flatten(),(R_A>tresh).values.flatten()  )for tresh in np.arange(0,1,0.01)]

In [ ]:
plt.plot(np.arange(0,1,0.01),nmis)

In [ ]:
df = get_kepler_plotable_data(segmentsMeta,speedDF,abs(R_A.fillna(0).values))
df.to_csv("../data/segment_flow_R.csv")

In [ ]:
df = get_kepler_plotable_data(segmentsMeta,speedDF,abs(L_A.fillna(0).values))
df.to_csv("../data/segment_flow_L.csv")

In [ ]:
A=R_A.fillna(0).values

In [ ]:
active_sum   = abs(A).sum(axis=0)

In [ ]:
active_count = (abs(A)!=0).sum(axis=0)

In [ ]:
active_mean  = active_sum/((active_count==0) + active_count)

In [ ]:
plt.plot(sorted(active_mean))
plt.figure()
plt.plot(sorted(active_sum))
plt.figure()
plt.plot(sorted(active_count))

In [ ]:
createMap(active_sum, log=True, name="active_sum_R")
plt.figure(figsize=(8,8))
plt.imshow(np.log1p(active_sum.reshape(1,-1)).T,  cmap=plt.cm.hot)
cbar=plt.colorbar()
cbar.ax.set_yticklabels(np.expm1( cbar.get_ticks()).round(2));
cbar.set_label("sum of active coefs")

In [ ]:
A=L_A.fillna(0).values

In [ ]:
active_sum   = abs(A).sum(axis=0)

In [ ]:
active_count = (abs(A)!=0).sum(axis=0)

In [ ]:
active_mean  = active_sum/((active_count==0) + active_count)

In [ ]:
plt.plot(sorted(active_mean))
plt.figure()
plt.plot(sorted(active_sum))
plt.figure()
plt.plot(sorted(active_count))

In [ ]:
createMap(active_sum, log=True, name="active_sum_L")
plt.figure(figsize=(8,8))
plt.imshow(np.log1p(active_sum.reshape(1,-1)).T,  cmap=plt.cm.hot)
cbar=plt.colorbar()
cbar.ax.set_yticklabels(np.expm1( cbar.get_ticks()).round(2));
cbar.set_label("sum of active coefs")

In [ ]:
4*sum(np.random.uniform(0,1,1000000)**2+np.random.uniform(0,1,1000000)**2<=1)/1000000